# Importação das bibliotecas

In [35]:
import pandas as pd
import seaborn as sns
import numpy as np
import plotly.express as px

In [74]:
def obter_resumo(tipo_imovel: str, bairro: str = None, banheiro: int = None, quartos: int = None):
    query = f'  tipo_imovel == "{tipo_imovel}"'
    if bairro is not None:
        query += f' and bairro_teste == "{bairro}"  '
    if banheiro is not None:
        query += f' and banheiro == {banheiro} '
    if quartos is not None:
        query += f' and quarto == {quartos} '
    base_original = pd.read_parquet('../data/processed/base_casa_v2.parquet')
    base_original = base_original.query(query)
    return base_original

In [113]:
def obter_estatistica(base_original : pd.DataFrame):
    print('Média preço: ', round(base_original['preco'].mean(), 2))
    print('Mediana Preço:',  base_original['preco'].median())
    print('Moda: ', base_original['preco'].mode()[0])

# Preço 

- Preço Geral

In [104]:
base_original = obter_resumo(tipo_imovel='Apartamento', bairro='Sumarezinho')
base_original

,tipo_imovel,nome,preco,metragem,quarto,banheiro,garagem,ID_CASA,bairro_teste
198,Apartamento,"Apartamento com 2 Quartos à Venda, 64m²",331000.0,64,2,2,1,2650686276,Sumarezinho
203,Apartamento,"Apartamento com 2 Quartos à Venda, 62m²",266000.0,62,2,1,1,2604957699,Sumarezinho
244,Apartamento,"Apartamento com 2 Quartos à Venda, 59m²",265000.0,59,2,2,1,2688462049,Sumarezinho
280,Apartamento,"Apartamento com 2 Quartos à Venda, 55m²",212000.0,55,2,1,1,2604952718,Sumarezinho
315,Apartamento,"Apartamento com 2 Quartos à Venda, 64m²",288000.0,64,2,2,1,2669958430,Sumarezinho
...,...,...,...,...,...,...,...,...,...
9003,Apartamento,"Apartamento com 2 Quartos à Venda, 68m²",211000.0,68,2,1,1,2673234280,Sumarezinho
9090,Apartamento,"Apartamento com 2 Quartos à Venda, 61m²",265000.0,61,2,2,1,2638381855,Sumarezinho
9358,Apartamento,"Apartamento com 2 Quartos à Venda, 58m²",265000.0,58,2,1,1,2697679401,Sumarezinho
9549,Apartamento,"Apartamento com 2 Quartos à Venda, 55m²",337000.0,55,2,2,1,2559856770,Sumarezinho


In [105]:
base_original['bairro_teste'].unique()

<StringArray>
['Sumarezinho']
Length: 1, dtype: string

In [106]:
n = base_original.shape[0]
k = 1 + (10 / 3) * np.log10(n)
k = int(k.round(0))

k

8

In [107]:
base_original.describe()

,preco,metragem,quarto,banheiro,garagem,ID_CASA
count,118.000000,118.000000,118.000000,118.000000,118.000000,1.180000e+02
mean,245524.661017,59.186441,2.127119,1.398305,1.042373,2.630640e+09
std,72993.168146,15.749616,0.359168,0.508724,0.378799,5.070566e+07
min,117000.000000,44.000000,2.000000,1.000000,0.000000,2.435754e+09
25%,195500.000000,47.000000,2.000000,1.000000,1.000000,2.599532e+09
50%,230000.000000,56.000000,2.000000,1.000000,1.000000,2.640957e+09
75%,270000.000000,64.000000,2.000000,2.000000,1.000000,2.669504e+09
max,590000.000000,123.000000,4.000000,3.000000,4.000000,2.698511e+09


In [108]:
base_original['preco_bins'] = pd.qcut(base_original['preco'], q=k).astype('str')

In [109]:
fig = px.box(base_original, x='preco_bins', y='preco', category_orders={})
fig.update_layout( xaxis={
                  'categoryorder': 'category ascending'})

fig.show()

In [114]:
obter_estatistica(base_original=base_original)

Média preço:  245524.66
Mediana Preço: 230000.0
Moda:  170000.0


In [111]:
fig = px.box(base_original, y='preco')
fig.show()

In [112]:
fig = px.histogram(base_original, x='preco', nbins=k)
fig.update_layout(bargap=0.2)
fig.show()